In [3]:
import pandas as pd

# Get S&P 500 company table from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)
df = tables[0]

# Check column for date added
if 'Date added' in df.columns:
    df['Date added'] = pd.to_datetime(df['Date added'], errors='coerce')
    df['Year added'] = df['Date added'].dt.year
    year_counts = df['Year added'].value_counts().drop(1957, errors='ignore')
    most_additions_year = year_counts.idxmax()
    print("Year with most additions:", most_additions_year)
else:
    print("No 'Date added' column found.")


Year with most additions: 2017


In [5]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 51.4 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.18.1-cp312-cp312-linux_x86_64.whl size=1029206 sha256=4fbecea2c073a9b9c21ea97f791dfe1700153622c8b5167288f82c72809e6eba
  Stored in directory: /home/codespace/.cache/pip/wheels/1a/57/6a/bb71346381d0d911cd4ce3026f1fa720da76707e4f01cf27dd
Successfully built peewee

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [8]:
import yfinance as yf

tickers = {
    'US': '^GSPC', 'China': '000001.SS', 'Hong Kong': '^HSI', 'Australia': '^AXJO',
    'India': '^NSEI', 'Canada': '^GSPTSE', 'Germany': '^GDAXI', 'UK': '^FTSE',
    'Japan': '^N225', 'Mexico': '^MXX', 'Brazil': '^BVSP'
}

returns = {}

for country, ticker in tickers.items():
    df = yf.download(ticker, start='2025-01-01', end='2025-05-01', progress=False, auto_adjust=True)
    
    if not df.empty and 'Close' in df.columns and len(df) > 1:
        try:
            start_price = float(df['Close'].iloc[0])
            end_price = float(df['Close'].iloc[-1])
            returns[country] = (end_price - start_price) / start_price
        except Exception as e:
            print(f"{country}: Error computing return: {e}")
    else:
        print(f"{country}: Data not available")

# Safely compare only if US return exists
us_return = returns.get('US', None)
if us_return is not None:
    better_than_us = [k for k, v in returns.items() if k != 'US' and isinstance(v, float) and v > us_return]
    
    print("\nS&P 500 YTD return:", round(us_return * 100, 2), "%")
    print("Indexes that performed better:", better_than_us)
    print("Count:", len(better_than_us))
else:
    print("Could not retrieve US return.")



S&P 500 YTD return: -5.1 %
Indexes that performed better: ['China', 'Hong Kong', 'Australia', 'India', 'Canada', 'Germany', 'UK', 'Mexico', 'Brazil']
Count: 9


/tmp/ipykernel_8334/3133867992.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  start_price = float(df['Close'].iloc[0])
/tmp/ipykernel_8334/3133867992.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  end_price = float(df['Close'].iloc[-1])
/tmp/ipykernel_8334/3133867992.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  start_price = float(df['Close'].iloc[0])
/tmp/ipykernel_8334/3133867992.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  end_price = float(df['Close'].iloc[-1])
/tmp/ipykernel_8334/3133867992.py:16: FutureWarning: Calling float on a single element Series is deprecate

In [8]:
url = 'https://stooq.com/q/d/l/?s=^spx&i=d'
df = pd.read_csv(url)
df.to_csv('data/sp500_stooq.csv', index=False)


In [10]:
import pandas as pd

# Load the data
df = pd.read_csv("data/sp500_stooq.csv", parse_dates=["Date"])
df = df.sort_values("Date").reset_index(drop=True)

# Keep only 'Date' and 'Close'
df = df[["Date", "Close"]].dropna()

# Ensure 'Close' is numeric
df["Close"] = pd.to_numeric(df["Close"], errors="coerce")

# Calculate All-Time High (ATH) and drawdowns
df["ATH"] = df["Close"].cummax()
df["Drawdown"] = (df["Close"] - df["ATH"]) / df["ATH"]

# Identify corrections >5% drawdown
corrections = []
in_correction = False
start_date = None

for i in range(1, len(df)):
    if not in_correction and df.loc[i, "Drawdown"] < -0.05:
        in_correction = True
        start_date = df.loc[i - 1, "Date"]
    elif in_correction and df.loc[i, "Close"] >= df.loc[i - 1, "ATH"]:
        end_date = df.loc[i, "Date"]
        duration = (end_date - start_date).days
        corrections.append(duration)
        in_correction = False

# Analyze durations
durations = pd.Series(corrections)
print(" Correction durations (in days):")
print("25th percentile:", durations.quantile(0.25))
print("Median:", durations.median())
print("75th percentile:", durations.quantile(0.75))


 Correction durations (in days):
25th percentile: 42.75
Median: 112.5
75th percentile: 537.0


In [14]:
import pandas as pd
import yfinance as yf

# Load and clean earnings data
df = pd.read_csv("data/ha1_Amazon.csv", sep=";")
df.rename(columns={
    "Earnings Date": "Date",
    "EPS Estimate": "Estimated EPS",
    "Reported EPS": "Actual EPS"
}, inplace=True)

# Fix date format with EDT time
df["Date"] = pd.to_datetime(df["Date"].str.replace(r"\s+at.*", "", regex=True), errors="coerce")
df["Surprise"] = df["Actual EPS"] > df["Estimated EPS"]
df = df.reset_index(drop=True)

# Download Amazon stock data
amzn = yf.download("AMZN", start="2010-01-01", end="2025-05-02", progress=False)
amzn["2d_return"] = amzn["Close"].shift(-2) / amzn["Close"] - 1
amzn = amzn.reset_index()

# Flatten MultiIndex if needed
if isinstance(amzn.columns, pd.MultiIndex):
    amzn.columns = amzn.columns.get_level_values(0)

# Merge
merged = pd.merge(df, amzn, on="Date", how="inner")

# Calculate medians
positive_median = merged[merged["Surprise"]]["2d_return"].median()
all_median = amzn["2d_return"].median()

print(f"Median 2-day return after positive earnings surprises: {positive_median:.4f}")
print(f" Median 2-day return across all days: {all_median:.4f}")


Median 2-day return after positive earnings surprises: 0.0201
 Median 2-day return across all days: 0.0022
